# Import packages

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class, process_GUAT_trials_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning
import seaborn as sns


plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



# Data for one session

In [ ]:
exists_ok = True
cgt = GUAT_vs_TAFT_class.GUATvsTAFTclass(ref_point_mode='time', 
                                            raw_data_folder_path='all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330',
                                            ref_point_value=-1.5,)
cgt.streamline_getting_GUAT_or_TAFT_x_df(GUAT_or_TAFT='TAFT', exists_ok=exists_ok)
cgt.streamline_getting_GUAT_or_TAFT_x_df(GUAT_or_TAFT='GUAT', exists_ok=exists_ok)

cgt.get_relevant_monkey_data(already_retrieved_ok=True)
cgt.find_patterns()
cgt.make_PlotTrials_args()
cgt.make_or_retrieve_target_clust_last_vis_df()


cgt.make_curvature_df()
cgt.find_opt_arc_info_from_curvature_df()
cgt.make_best_arc_df()
cgt.get_elements_for_plotting()

# get null arc info for plotting
cgt.null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(cgt.best_arc_df, cgt.monkey_information, column_for_color=None, ignore_error=True)
all_point_index = cgt.best_arc_df.point_index.values
all_ff_index = cgt.best_arc_df.ff_index.values
remaining_index_of_array, remaining_all_ff_index, remaining_all_point_index = show_null_trajectory.eliminate_invalid_ff_for_null_arc(all_ff_index, all_point_index, cgt.ff_real_position_sorted, cgt.monkey_information)
cgt.find_and_package_arc_to_center_info_for_plotting(remaining_all_point_index, remaining_all_ff_index)

pretty_null_arc_plot_kwargs = {'player': 'monkey',
        'show_believed_target_positions': True,                       
        'show_stops': True,
        'show_reward_boundary': True,
        'show_scale_bar': True,
        'hitting_arena_edge_ok': True,
        'trial_too_short_ok': True,
        'show_alive_fireflies': False,
        'adjust_xy_limits': True,
        'zoom_in': True,
        'minimal_margin': 60,
        }


### plot trials

In [ ]:
GUAT_or_TAFT = 'GUAT'
list_of_stop_point_index = cgt.GUAT_x_df[cgt.GUAT_x_df['cur_ff_distance_at_ref'] > 150].stop_point_index.values

In [ ]:
GUAT_or_TAFT = 'TAFT'
list_of_stop_point_index = cgt.TAFT_x_df[cgt.TAFT_x_df['cur_ff_distance_at_ref'] > 150].stop_point_index.values

In [ ]:
sub = cgt.take_out_subsets_to_plot(list_of_stop_point_index, GUAT_or_TAFT)

In [ ]:
# counter = 0
# for i, row in sub.iterrows():
#     if GUAT_or_TAFT == 'GUAT':
#         ff_column = 'latest_visible_ff'
#     else:
#         ff_column = 'ff_index'

#     print('cur_ff_distance_at_ref', row['cur_ff_distance_at_ref'])

    
#     plotting_kwargs_temp = copy.deepcopy(cgt.plotting_kwargs)
#     plotting_kwargs_temp['indices_of_ff_to_mark'] = [row[ff_column]]
#     trial = row['trial']
#     duration = [cgt.ff_caught_T_new[trial-2], cgt.ff_caught_T_new[trial]+1]
#     returned_info = plot_trials.PlotTrials(duration, 
#                 *cgt.PlotTrials_args,
#                 **plotting_kwargs_temp,         
#                 )
#     plt.show()
#     # process_GUAT_trials_class.ProcessGUATtrials.make_GUAT_plot(cgt, trial, [row[ff_column]], np.arange(row['stop_point_index']-200, row['stop_point_index']+300),
#     #                                                            additional_kwargs=additional_kwargs)
#     if counter == 3:
#         break
#     counter += 1

# Exp

In [ ]:
try_a_few_times_trials = cgt.try_a_few_times_trials
give_up_after_trying_trials = cgt.give_up_after_trying_trials

target_cluster_TAFT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(try_a_few_times_trials)]
target_cluster_GUAT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(give_up_after_trying_trials)]

sort_1_df = target_cluster_TAFT
sort_1_trials = sort_1_df.target_index.values
sort_1_name = "Try a few times"

sort_2_df = GUAT_item.GUAT_cluster_df
sort_2_trials = sort_2_df.target_index.values
sort_2_name = "Give up after trying"
sort_2_ff_indices = GUAT_item.GUAT_cluster_df.last_vis_ff_index.values

category_item = category_class.ProcessCategoryData(sort_1_trials=sort_1_trials, sort_2_trials=sort_2_trials,
                                                              sort_1_name=sort_1_name, sort_2_name=sort_2_name, 
                                                              sort_1_df=sort_1_df, sort_2_df=sort_2_df,
                                                              sort_2_ff_indices = sort_2_ff_indices,
                                                              sort_2_ff_positions = cgt.ff_real_position_sorted[sort_2_ff_indices],                                                             
                                                              PlotTrials_args=cgt.PlotTrials_args, ff_flash_sorted=cgt.ff_flash_sorted,
                                                              null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)

In [ ]:
target_cluster_TAFT.columns

In [ ]:
'time_since_last_vis', 'last_vis_dist', 'last_vis_cum_dist', 'last_vis_ang', 'last_vis_ang_to_bndry', 'abs_last_vis_ang', 'abs_last_vis_ang_to_bndry'

# Exp 2

In [ ]:
exists_ok = True
cgt = GUAT_vs_TAFT_class.GUATvsTAFTclass(ref_point_mode='time', 
                                            raw_data_folder_path='all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330',
                                            ref_point_value=-1.5,)
cgt.make_decision_making_basic_ff_info()
decision_making_basic_ff_info_cleaned = cgt.decision_making_basic_ff_info_cleaned.copy()

In [ ]:
prob_sub = cgt.decision_making_basic_ff_info.loc[(cgt.decision_making_basic_ff_info['whether_switched'] == 1) 
                                      & (cgt.decision_making_basic_ff_info['ff_distance'] < 25)
                                      & (cgt.decision_making_basic_ff_info['time_since_ff_last_seen'] < 2)
                                      ]
prob_sub['stop_time'].values

In [ ]:
cgt.plotting_kwargs = {'player': 'monkey',
                       'show_stops': True,
                       'show_believed_target_positions': True,
                       'show_reward_boundary': True,
                       'show_scale_bar': True,
                       'hitting_arena_edge_ok': True,
                       'trial_too_short_ok': True,
                       'show_connect_path_ff': False,
                       'vary_color_for_connecting_path_ff': False,
                       'show_connect_path_ff_memory': False,
                       'show_alive_fireflies': False,
                       'show_visible_fireflies': True,
                       'show_in_memory_fireflies': True,
                       'connect_path_ff_max_distance': 400,
                       'zoom_in': True,
                       'minimal_margin': 40,
                       }

In [ ]:
counter = 0
for index, row in prob_sub.iterrows():
    plotting_kwargs_temp = copy.deepcopy(cgt.plotting_kwargs)
    plotting_kwargs_temp['indices_of_ff_to_mark'] = [int(row['ff_index'])]
    plotting_kwargs_temp['point_indices_to_be_marked'] = [row['stop_point_index']]

    duration = [row['stop_time'] - 1, row['stop_time'] + 1]
    returned_info = plot_trials.PlotTrials(duration, 
                *cgt.PlotTrials_args,
                **plotting_kwargs_temp,         
                )
    plt.show()
    # process_GUAT_trials_class.ProcessGUATtrials.make_GUAT_plot(cgt, trial, [row[ff_column]], np.arange(row['stop_point_index']-200, row['stop_point_index']+300),
    #                                                            additional_kwargs=additional_kwargs)
    
    
    if counter == 3:
        break
    counter += 1

# Try a few times vs. give up after trying

## Compared with the rest

In [ ]:
reload(decision_making_class)
reload(plot_trials)
reload(intended_targets_classes)

try_a_few_times_trials = cgt.try_a_few_times_trials
target_cluster_TAFT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(try_a_few_times_trials)]
sort_1_df = target_cluster_TAFT
sort_1_trials = sort_1_df.target_index.values
sort_1_name = "Try a few times"


target_cluster_no_flash_after_stopn_TAFT = cgt.target_clust_last_vis_df[~cgt.target_clust_last_vis_df['target_index'].isin(try_a_few_times_trials)]
sort_2_df = target_cluster_no_flash_after_stopn_TAFT
sort_2_trials = sort_2_df.target_index.values
sort_2_name = "Non-TAFT"

category_item = category_class.ProcessCategoryData(sort_1_trials=sort_1_trials, sort_2_trials=sort_2_trials,
                                                              sort_1_name=sort_1_name, sort_2_name=sort_2_name, 
                                                              sort_1_df=sort_1_df, sort_2_df=sort_2_df,                                                     
                                                              PlotTrials_args=cgt.PlotTrials_args, ff_flash_sorted=cgt.ff_flash_sorted,
                                                              null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)

In [ ]:
reload(show_null_trajectory)
reload(decision_making_class)
reload(plot_trials)
reload(intended_targets_classes)
reload(category_class)


category_item.clean_out_cross_boundary_trials()
category_item.clean_out_trials_where_target_cluster_was_not_seen_for_a_long_time_before_capture()
category_item.make_polar_plot_of_target_last_seen_positions()
category_item.make_histograms_of_target_last_seen_attributes()
category_item.make_histogram_of_distances_from_previous_targets()
category_item.make_polar_plot_of_positions_from_previous_targets()


category_item.plot_trajectories(trials=category_item.sort_1_trials[17:18], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)
category_item.plot_distributions_of_visible_ff_and_in_memory_ff()


## visualize predictions

In [ ]:
gnb = GaussianNB()
# gnb.fit(ml_inst.x_var_df, ml_inst.y_var_df)
#(trained_model = gnb, max_plot_to_make = 2)

#category_item.inspect_special_cases(weird_trials=[98, 180, 212, 649])

## plot_trajectories

In [ ]:
current_i = 30

In [ ]:
reload(specific_utils)
reload(plot_trials)
category_item.plot_trajectories(trials=category_item.sort_1_trials[current_i:current_i+2], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)
current_i = current_i+2

## 724

In [ ]:
cgt.null_arc_to_center_info_for_plotting[cgt.null_arc_to_center_info_for_plotting['arc_point_index']==112997]

In [ ]:
pretty_null_arc_plot_kwargs['show_ff_indices'] = False

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1856.504211, 1862.693883]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([908, 723, 724])
intended_target_ff_index = None #749
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


# For ppt

## Exp

In [ ]:
sort_1_trials = sort_1_df.target_index.values
sort_1_name = "Try a few times"

sort_2_df = GUAT_item.GUAT_cluster_df
sort_2_trials = sort_2_df.target_index.values

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]

for trial in sort_1_trials[60:80]:
    duration_to_plot = [cgt.ff_caught_T_new[trial] - 3, cgt.ff_caught_T_new[trial] + 1]

    max_num_plot_to_make = 1
    ff_indices_to_plot_null_arc = np.array([trial]) #np.array([908, 723, 724])
    intended_target_ff_index = None #749
    point_indices_to_plot_null_arc = []
    pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
    pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
    #pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
    pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
    pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
    pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

    pretty_null_arc_plot_kwargs_temp.update({'show_start': False})

    time_to_begin_plotting_null_arc = None
    time_between_every_two_null_arcs = 0.3

    updated_args = {'title': ''}


    show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                            plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                            show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                            **updated_args)


## TAFT (retry)

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1856.504211, 1862.693883]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([908, 723, 724])
intended_target_ff_index = None #749
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

pretty_null_arc_plot_kwargs_temp.update({'show_start': False})

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3

updated_args = {'title': ''}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        **updated_args)


## GUAT (switch)

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1537.023148, 1542.046287]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([598, 599, 627])
intended_target_ff_index = 717
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

pretty_null_arc_plot_kwargs_temp.update({'show_start': False})

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3

updated_args = {'title': ''}

show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        **updated_args)



# Give up after trying

## GUAT_cluster_df

In [ ]:
## if I want to use manual_anno for GUAT_item
# dm = decision_making_class.DecisionMaking(raw_data_folder_path=raw_data_folder_path)
# dm.retrieve_manual_anno()
# manual_anno = dm.manual_anno

In [ ]:
reload(process_GUAT_trials_class)
auto_annot, auto_annot_long = decision_making_utils.make_auto_annot(cgt.best_arc_df, cgt.monkey_information, cgt.ff_caught_T_new)

GUAT_item = process_GUAT_trials_class.ProcessGUATtrials(cgt.give_up_after_trying_info_bundle, \
                                                        cgt.PlotTrials_args, max_distance_to_stop_for_GUAT_target=50, max_allowed_time_since_last_vis=2.5)
GUAT_item.find_possible_objects_of_pursuit_in_GUAT()
GUAT_item.find_GUAT_ff_aimed_at_from_manual_anno(auto_annot)
#GUAT_item.check_GUAT_object_with_manual_anno(verbose=False)
#GUAT_item.inspect_clusters_w_o_matching_ff()
GUAT_item.make_GUAT_cluster_df()

## Compare with TAFT

In [ ]:
cgt.raw_data_folder_name

In [ ]:
try_a_few_times_trials = cgt.try_a_few_times_trials
give_up_after_trying_trials = cgt.give_up_after_trying_trials

target_cluster_TAFT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(try_a_few_times_trials)]
target_cluster_GUAT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(give_up_after_trying_trials)]

sort_1_df = target_cluster_TAFT
sort_1_trials = sort_1_df.target_index.values
sort_1_name = "Retry + Capture" #"Try a few times"

sort_2_df = GUAT_item.GUAT_cluster_df
sort_2_trials = sort_2_df.target_index.values
sort_2_name = "Retry + Switch" #"Give up after trying"
sort_2_ff_indices = GUAT_item.GUAT_cluster_df.last_vis_ff_index.values

category_item = category_class.ProcessCategoryData(sort_1_trials=sort_1_trials, sort_2_trials=sort_2_trials,
                                                              sort_1_name=sort_1_name, sort_2_name=sort_2_name, 
                                                              sort_1_df=sort_1_df, sort_2_df=sort_2_df,
                                                              sort_2_ff_indices = sort_2_ff_indices,
                                                              sort_2_ff_positions = cgt.ff_real_position_sorted[sort_2_ff_indices],                                                             
                                                              PlotTrials_args=cgt.PlotTrials_args, ff_flash_sorted=cgt.ff_flash_sorted,
                                                              null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)

In [ ]:

category_item.clean_out_cross_boundary_trials()
category_item.clean_out_trials_where_target_cluster_was_not_seen_for_a_long_time_before_capture()
category_item.make_polar_plot_of_target_last_seen_positions()
category_item.make_histograms_of_target_last_seen_attributes()
# category_item.make_histogram_of_distances_from_previous_targets()
# category_item.make_polar_plot_of_positions_from_previous_targets()

# category_item.plot_trajectories(trials=category_item.sort_1_trials[17:18], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)
# category_item.plot_distributions_of_visible_ff_and_in_memory_ff()


## the below function (free selection prediction) shouldn't be used on GUAT. So far, it should only be used on VBLO.
# category_item.make_and_visualize_free_selection_predictions_using_trained_model(trained_model = gnb, max_plot_to_make = 99)
# category_item.inspect_special_cases(weird_trials=[98, 180, 212, 649])

## heading

In [ ]:
ps = monkey_plan_factors_x_sess_class.PlanAcrossSessions(monkey_name='monkey_Schro')
monkey_median_df = ps.make_or_retrieve_all_ref_pooled_median_info(exists_ok=True, 
                                                                pooled_median_info_exists_ok=True)

monkey_median_df['diff_in_abs_angle_to_nxt_ff_median'] = monkey_median_df['diff_in_abs_angle_to_nxt_ff_median'] / 180 * np.pi
monkey_median_df['diff_in_abs_angle_to_nxt_ff_boot_med_std'] = monkey_median_df['diff_in_abs_angle_to_nxt_ff_boot_med_std'] / 180 * np.pi
ps.plot_heading_in_all_ref_median_info(monkey_median_df)



## curv

In [ ]:
# Difference in Curvature Across Two Arc Pairs

In [ ]:
ps = monkey_plan_factors_x_sess_class.PlanAcrossSessions(monkey_name='monkey_Schro')
monkey_median_df = ps.make_or_retrieve_all_ref_pooled_median_info(exists_ok=True, 
                                                                pooled_median_info_exists_ok=True)
ps.plot_curv_in_all_ref_median_info()



## perc

In [ ]:
from planning_analysis.factors_vs_indicators import make_variations_utils, _predict_y_values_class, _compare_y_values_class, _plot_variations_class

In [ ]:
reload(_plot_variations_class)
reload(plot_variations_utils)

In [ ]:
ps = monkey_plan_factors_x_sess_class.PlanAcrossSessions(monkey_name='monkey_Schro')
monkey_perc_df = ps.make_or_retrieve_pooled_perc_info(exists_ok=True)
ps.plot_direction_in_perc_info()
ps.fig.show()


In [ ]:
for monkey in ['Bruno', 'Schro']:
    ps = monkey_plan_factors_x_sess_class.PlanAcrossSessions(monkey_name=f'monkey_{monkey}')
    monkey_perc_df = ps.make_or_retrieve_pooled_perc_info(exists_ok=True)
    ps.plot_direction_in_perc_info()
    


    ps.fig.update_layout(
                    title=dict(
                    text=f"{monkey}: Same-Side Stop Rate",
                    x=0.5,         # center title
                    y=0.95,        # a little below the top edge
                    xanchor="center",
                    yanchor="top",
                    font=dict(size=18)
                ),
        yaxis=dict(
            ticksuffix="%",   # just add % symbol to numbers
            title="Percentage (Median ± 2 Bootstrap SE)"
        )
    )
    ps.fig.show()

## heading

In [ ]:
ps = monkey_plan_factors_x_sess_class.PlanAcrossSessions(monkey_name='monkey_Schro')
monkey_median_df = ps.make_or_retrieve_all_ref_pooled_median_info(exists_ok=True, 
                                                                pooled_median_info_exists_ok=True)

monkey_median_df['diff_in_abs_angle_to_nxt_ff_median'] = monkey_median_df['diff_in_abs_angle_to_nxt_ff_median'] / 180 * np.pi
monkey_median_df['diff_in_abs_angle_to_nxt_ff_boot_med_std'] = monkey_median_df['diff_in_abs_angle_to_nxt_ff_boot_med_std'] / 180 * np.pi
ps.plot_heading_in_all_ref_median_info(monkey_median_df)



## curv

In [ ]:
# Difference in Curvature Across Two Arc Pairs

In [ ]:
ps = monkey_plan_factors_x_sess_class.PlanAcrossSessions(monkey_name='monkey_Schro')
monkey_median_df = ps.make_or_retrieve_all_ref_pooled_median_info(exists_ok=True, 
                                                                pooled_median_info_exists_ok=True)
ps.plot_curv_in_all_ref_median_info()



## perc

In [ ]:
from planning_analysis.factors_vs_indicators import make_variations_utils, _predict_y_values_class, _compare_y_values_class, _plot_variations_class

In [ ]:
reload(_plot_variations_class)
reload(plot_variations_utils)

In [ ]:
ps = monkey_plan_factors_x_sess_class.PlanAcrossSessions(monkey_name='monkey_Schro')
monkey_perc_df = ps.make_or_retrieve_pooled_perc_info(exists_ok=True)
ps.plot_direction_in_perc_info()
ps.fig.show()


In [ ]:
for monkey in ['Bruno', 'Schro']:
    ps = monkey_plan_factors_x_sess_class.PlanAcrossSessions(monkey_name=f'monkey_{monkey}')
    monkey_perc_df = ps.make_or_retrieve_pooled_perc_info(exists_ok=True)
    ps.plot_direction_in_perc_info()
    


    ps.fig.update_layout(
                    title=dict(
                    text=f"{monkey}: Same-Side Stop Rate",
                    x=0.5,         # center title
                    y=0.95,        # a little below the top edge
                    xanchor="center",
                    yanchor="top",
                    font=dict(size=18)
                ),
        yaxis=dict(
            ticksuffix="%",   # just add % symbol to numbers
            title="Percentage (Median ± 2 Bootstrap SE)"
        )
    )
    ps.fig.show()

## Compared with the rest

In [ ]:
give_up_after_trying_trials = cgt.give_up_after_trying_trials

target_cluster_no_flash_after_stopn_GUAT = cgt.target_clust_last_vis_df[~cgt.target_clust_last_vis_df['target_index'].isin(give_up_after_trying_trials)]
target_cluster_GUAT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(give_up_after_trying_trials)]

sort_1_df = target_cluster_no_flash_after_stopn_GUAT
sort_1_trials = sort_1_df.target_index.values
sort_1_name = "Non-GUAT"

sort_2_df = GUAT_item.GUAT_cluster_df
sort_2_trials = sort_2_df.target_index.values
sort_2_name = "Give up after trying"
sort_2_ff_indices = GUAT_item.GUAT_cluster_df.last_vis_ff_index.values
sort_2_time_for_predicting_ff = GUAT_item.GUAT_cluster_df.latest_visible_time_before_last_stop.values

category_item = category_class.ProcessCategoryData(sort_1_trials=sort_1_trials, sort_2_trials=sort_2_trials,
                                                              sort_1_name=sort_1_name, sort_2_name=sort_2_name, 
                                                              sort_1_df=sort_1_df, sort_2_df=sort_2_df,
                                                              sort_2_ff_indices = sort_2_ff_indices,
                                                              sort_2_ff_positions = cgt.ff_real_position_sorted[sort_2_ff_indices],   
                                                              sort_2_time_for_predicting_ff = sort_2_time_for_predicting_ff,                                                       
                                                              PlotTrials_args=cgt.PlotTrials_args, ff_flash_sorted=cgt.ff_flash_sorted,
                                                              null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)

In [ ]:
category_item.clean_out_cross_boundary_trials()
category_item.clean_out_trials_where_target_cluster_was_not_seen_for_a_long_time_before_capture()
category_item.make_polar_plot_of_target_last_seen_positions()
category_item.make_histograms_of_target_last_seen_attributes()
category_item.make_histogram_of_distances_from_previous_targets()
category_item.make_polar_plot_of_positions_from_previous_targets()
category_item.plot_trajectories(trials=category_item.sort_1_trials[17:18], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)
category_item.plot_distributions_of_visible_ff_and_in_memory_ff()
## The function below (free selection prediction) shouldn't be used on GUAT. So far, it should only be used on VBLO.
#category_item.make_and_visualize_free_selection_predictions_using_trained_model(trained_model = gnb, use_sort_1=False, use_sort_2=True, max_plot_to_make=3)
# category_item.inspect_special_cases(weird_trials=[98, 180, 212, 649])

## plot_trajectories

In [ ]:
current_i = 60

In [ ]:
reload(specific_utils)
reload(plot_trials)
cgt.null_arc_info_for_plotting_temp = cgt.null_arc_info_for_plotting.copy()
cgt.null_arc_info_for_plotting_temp['show_connect_path_ff'] = True
category_item.plot_trajectories(trials=category_item.sort_2_trials[current_i:current_i+2], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting_temp)
current_i = current_i+2

## 489 (Bruno 0330)

In [ ]:

reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1254, 1262]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1042, 1241, 489, 685, 744])
intended_target_ff_index = 1241
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, show_ff_indices=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 567 ??

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
# duration_to_plot = [1469.670162, 1475.0996949999999]
duration_to_plot = [1471, 1475.0996949999999]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([780, 703, 655, 567])
intended_target_ff_index = None #1241
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 575 ?

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1483.929318, 1489.226027]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([575, 576, 624, 1620, 1170, 1351])
intended_target_ff_index = None #1241
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, show_ff_indices=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 586

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1504.795097, 1512.714639]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([595])
intended_target_ff_index = 595
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 599 ??

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1537.023148, 1542.046287]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([598, 599, 627])
intended_target_ff_index = 717
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )



## 622

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1594.507002, 1597.611809]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1351, 831, 856])
intended_target_ff_index = 856
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 1
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 636

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
# duration_to_plot = [1625.433097, 1629.310465]
duration_to_plot = [1621.433097, 1629.310465]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1464, 878, 636, 974])
intended_target_ff_index = 636
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 641

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1643, 1655]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([726, 665])
intended_target_ff_index = 726
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 641 (2) x

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1646, 1652]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([822, 641, 665])
intended_target_ff_index = 822
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 650 ?

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1666.310034, 1671.855963]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1089, 694, 650])
intended_target_ff_index = None#822
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 653

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1674.543492, 1681.62897]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([708, 660, 820])
intended_target_ff_index = 820
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 653 (2)

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1677.543492, 1683.62897]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([708, 690, 771, 653, 789])
intended_target_ff_index = 708
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 658 (too hard to see)

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1684.973395, 1692.555617]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([668, 656, 657, 704, 797, 798, 824, 658])
intended_target_ff_index = None # 658
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 660 

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1692.487749, 1697.950517]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([751, 660, 820, 661])
intended_target_ff_index = 751
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 679

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1752.16371, 1761.527301]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = None #np.array()
intended_target_ff_index = 728
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=450, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 693

In [ ]:
 [sns.color_palette("tab10", 3), sns.color_palette("tab10", 14)[4:]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define palettes

palette2 = sns.color_palette("tab10", 16)

# Concatenate
combined_palette =  palette2

# Visualize as swatches
sns.palplot(combined_palette)
plt.show()


In [ ]:
# Define blue and pink (as RGB tuples in [0,1])
custom_colors = [(0, 0, 1), (1, 0.75, 0.8)]  # blue, pink
# Define your existing palettes
palette1 = sns.color_palette("tab10", 3)
palette2 = sns.color_palette("tab10", 14)[4:]
# Combine
varying_colors = np.concatenate(
    [custom_colors, palette1, palette2], axis=0
)
print(varying_colors.shape)  # should be (15, 3) since 2 + 3 + 10
    

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
# duration_to_plot = [1791.986039, 1798.079852]
duration_to_plot = [1793.5, 1798.079852]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([692, 693, 724])
intended_target_ff_index = 724
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'


time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=450, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        title=''
                                        )


## 713

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1831, 1836.149079]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = None #np.array([692, 693, 724])
intended_target_ff_index = 1492
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=330, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 718

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1838, 1843.994575]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([717, 1401, 718, 845, 905])
intended_target_ff_index = 1401
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 723

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1849.504211, 1854.4]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([722, 749])
intended_target_ff_index = 749
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 730

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1873.116986, 1878.281488]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1106])
intended_target_ff_index = 1106
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 751

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1919.422092, 1926.644391]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([876, 751])
intended_target_ff_index = 876
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 756

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1934.510997, 1943.393497]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([756, 1089, 1703])
intended_target_ff_index = 1089
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3

updated_args = {'title': ''}

show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        **updated_args)


## 765

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1959.975111, 1968.49221]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = None #np.array([756, 1089, 1703])
intended_target_ff_index = None #1089
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


In [ ]:
category_item.plot_trajectories(trials=[765], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting,
                                show_connect_path_ff=True, show_ff_indices=True)

In [ ]:
reload(show_null_trajectory)
duration_to_plot = [1254.09006, 1260.797728]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = None #np.array([62, 413, 60, 61])
intended_target_ff_index = 1241
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=250, ff_max_distance_to_path=400, time_between_every_two_null_arcs=0.2, time_to_begin_plotting_null_arc=None, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

In [ ]:
STOP!

# Appendix

## Plot trials

In [ ]:
classic_plot_kwargs = {'show_stops': True,
                       'show_believed_target_positions': True,
                       'show_reward_boundary': True,
                       'show_scale_bar': True}
cgt.PlotTrials_args = (cgt.monkey_information, cgt.ff_dataframe, cgt.ff_life_sorted, cgt.ff_real_position_sorted, cgt.ff_believed_position_sorted, cgt.cluster_around_target_indices, cgt.ff_caught_T_new)

In [ ]:
from data_wrangling import general_utils

In [ ]:

num_trials = 2 


with general_utils.initiate_plot(10,10,100):
  for currentTrial in [96+1, 102+1]: #two_in_a_row[10:20]:
      duration = [cgt.ff_caught_T_new[currentTrial-num_trials], cgt.ff_caught_T_new[currentTrial]]
      plot_trials.PlotTrials(duration, 
                  *cgt.PlotTrials_args,
                  **classic_plot_kwargs,
                  currentTrial = currentTrial,
                  num_trials = num_trials,                   
                  show_connect_path_ff_except_targets = True,
                  show_path_when_target_visible = True,
                  show_ff_indices = False,
                  hitting_arena_edge_ok = True,
                  visible_distance = 400,
                  minimal_margin = 300,
                  show_monkey_angles = True
                  )
      plt.show()  

      